# M5 - Forecasting Competition - Accuracy

**This will be our first notebook to send a simple submission with raw features to understand how data is structured, fields necessary and basic steps to create a prediction.**

Taken from competitors guide directly:

The M5 dataset, generously made available by Walmart, involves the unit sales of various products sold in the USA, organized in the form of grouped time series. More specifically, the dataset involves the unit sales of 3,049 products, classified in 3 product categories (Hobbies, Foods, and Household) and 7 product departments, in which the above-mentioned categories are disaggregated.  The products are sold across ten stores, located in three States (CA, TX, and WI). You can read more about it in [competitor's guide](../references/m5-competitors-guide.docx)

### Data Dictionary for Quick Check

File 1: “calendar.csv” 
Contains information about the dates the products are sold.
- date: The date in a “y-m-d” format.
- wm_yr_wk: The id of the week the date belongs to.
- weekday: The type of the day (Saturday, Sunday, …, Friday).
- wday: The id of the weekday, starting from Saturday.
- month: The month of the date.
- year: The year of the date.
- event_name_1: If the date includes an event, the name of this event.
- event_type_1: If the date includes an event, the type of this event.
- event_name_2: If the date includes a second event, the name of this event.
- event_type_2: If the date includes a second event, the type of this event.
- snap_CA, snap_TX, and snap_WI: A binary variable (0 or 1) indicating whether the stores of CA, TX or WI allow SNAP  purchases on the examined date. 1 indicates that SNAP purchases are allowed.

File 2: “sell_prices.csv”
Contains information about the price of the products sold per store and date.
- store_id: The id of the store where the product is sold. 
- item_id: The id of the product.
- wm_yr_wk: The id of the week.
- sell_price: The price of the product for the given week/store. The price is provided per week (average across seven days). If not available, this means that the product was not sold during the examined week. Note that although prices are constant at weekly basis, they may change through time (both training and test set).

File 3: “sales_train.csv” 
Contains the historical daily unit sales data per product and store.
- item_id: The id of the product.
- dept_id: The id of the department the product belongs to.
- cat_id: The id of the category the product belongs to.
- store_id: The id of the store where the product is sold.
- state_id: The State where the store is located.
- d_1, d_2, …, d_i, … d_1941: The number of units sold at day i, starting from 2011-01-29. 


### Evaluation - Quick Check

#### Forecasting horizon

The number of forecasts required, both for point and probabilistic forecasts, is h=28 days (4 weeks ahead). 

The performance measures are **first computed for each series** separately by averaging their values across the forecasting horizon and **then averaged again across the series** in a weighted fashion (see below) to obtain the final scores.


#### Point Forecasts

The accuracy of the point forecasts will be evaluated using the Root Mean Squared Scaled Error (RMSSE), which is a variant of the well-known Mean Absolute Scaled Error (MASE) proposed by Hyndman and Koehler (2006) . The measure is calculated for each series as follows:

$$ RMSSE = \sqrt{\frac{1}{h}\frac{\sum_{t = n+1}^{n+h} (Y_{t} - \hat{Y_{t}})^{2}}{\frac{1}{n-1}\sum_{t = 2}^{n} (Y_{t} - {Y_{t-1}})^{2}}} $$

where $ Y_{t} $ is the actual future value, $ \hat{Y_{t}} $ is the predicted future value, n is the length of the training sampe and h the forecasting horizon. Intuitively it calculate how well a forecast will be regarding a naive forecasting applied to in-sample data.

*Justifications*:

 - Since we have intermittent data with a lot of zeros, a MASE approach would optimize forecast for the median, which is closer to zero. However, since major goal is to accurately forecast average demand, they choosed a metric based on squared erros, which are optimized for the mean.
 - The measure is scale independent, which is best suited for series in different scales.
 - It can be safely computed as does not rely on divisions with values that could be equal or close to zero.
 - The measure penalizes positive and negative forecast errors, as well as large and small forecasts, equally, thus being symmetric.
 
 
After estimating the RMSSE for all the 42,840 time series of the competition, the participating methods will be ranked using the Weighted RMSSE (WRMSSE), as described latter in this Guide, using the following formula:


$$ WRMSSE = \sum_{i=1}^{42,840} w_{i}*RMSSE $$

where $ w_{i} $ is the weight of the $ i_{th} $ series of the competition. A lower WRMSSE score is better.

## Importings

### Importing Libraries

In [5]:
import sys

sys.path.append("..")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor

from common import reduce_mem_usage

### Importing Data

In [7]:
df_sales = pd.read_csv("../data/raw/sales_train_validation.csv")
df_sales.head(5)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [8]:
df_sample_sub = pd.read_csv("../data/raw/sample_submission.csv")
df_sample_sub.head(5)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df_calendar = pd.read_csv("../data/raw/calendar.csv")
df_calendar.head(5)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [10]:
df_prices = pd.read_csv("../data/raw/sell_prices.csv")
df_prices.head(5)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


## Baseline Models

### Really First Submission

I just want to submit data to understand how is the format and transformations necessary.

In [11]:
df_sales_melted = (df_sales.melt(id_vars=["id", "item_id", "dept_id", "store_id", "state_id", "cat_id"],
                                    var_name="d",
                                    value_name="unit_solds")
                            .assign(simple_trend=lambda df: df["d"].str.extract(r"(\d+)").astype(int))
                    )

In [12]:
df_sales_melted.head(6)

,id,item_id,dept_id,store_id,state_id,cat_id,d,unit_solds,simple_trend
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,CA_1,CA,HOBBIES,d_1,0,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,CA_1,CA,HOBBIES,d_1,0,1
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,CA_1,CA,HOBBIES,d_1,0,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,CA_1,CA,HOBBIES,d_1,0,1
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,CA_1,CA,HOBBIES,d_1,0,1
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,CA_1,CA,HOBBIES,d_1,0,1


In [13]:
df_sales_cal = (df_sales_melted.merge(df_calendar,
                                        on="d",
                                        how="left"))

In [14]:
df_sales_cal.tail(4)

,id,item_id,dept_id,store_id,state_id,cat_id,d,unit_solds,simple_trend,date,...,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
58327366,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,WI_3,WI,FOODS,d_1913,0,1913,2016-04-24,...,2,4,2016,NaN,NaN,NaN,NaN,0,0,0
58327367,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,WI_3,WI,FOODS,d_1913,0,1913,2016-04-24,...,2,4,2016,NaN,NaN,NaN,NaN,0,0,0
58327368,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,WI_3,WI,FOODS,d_1913,3,1913,2016-04-24,...,2,4,2016,NaN,NaN,NaN,NaN,0,0,0
58327369,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,WI_3,WI,FOODS,d_1913,0,1913,2016-04-24,...,2,4,2016,NaN,NaN,NaN,NaN,0,0,0


In [17]:
df_all_features =( df_sales_cal
                   .merge(df_prices, 
                          on=["item_id", "store_id", "wm_yr_wk"], 
                          how="left")
                  .pipe(reduce_mem_usage))
df_all_features.shape

Mem. usage decreased to 7064.41 Mb (33.9% reduction)


(58327370, 23)

In [18]:
categorical_cols = ["item_id", "dept_id", "store_id", "state_id", "cat_id", "event_name_1", "event_name_2",
                    "event_type_1", "event_type_2"]

for categorical_col in categorical_cols:
    if "event" in categorical_col:
        df_all_features[categorical_col] = df_all_features[categorical_col].fillna("NULL")
        
    lb = LabelEncoder()
    fname = f"{categorical_col}_enc"
    df_all_features[fname] = lb.fit_transform(df_all_features[categorical_col])

In [19]:
lgmb_clf = LGBMRegressor(colsample_bytree=0.7, max_depth=7)

In [20]:
target= ['unit_solds']

categorical_enc_features = [feature+"_enc" for feature in categorical_cols]
numerical_features = ["simple_trend", "wday", "month", "snap_CA", "snap_TX", "snap_WI", "sell_price"]
features = numerical_features + categorical_enc_features

In [21]:
lgmb_clf.fit(df_all_features[features], df_all_features[target]);

In [22]:
df_sample_sub["cat_id"] = df_sample_sub["id"].str.split("_").apply(lambda x: x[0])
df_sample_sub["item_id"] = df_sample_sub["id"].str.split("_").apply(lambda x: '_'.join(x[:3]))
df_sample_sub["dept_id"] = df_sample_sub["id"].str.split("_").apply(lambda x: '_'.join(x[:2]))
df_sample_sub["store_id"] = df_sample_sub["id"].str.split("_").apply(lambda x: '_'.join(x[3:5]))
df_sample_sub["state_id"] = df_sample_sub["id"].str.split("_").apply(lambda x: x[3])

In [38]:
df_sample_melted = (df_sample_sub
                      .melt(id_vars=["id", "item_id", "dept_id", "store_id", "state_id", "cat_id"],
                                                        var_name="date",
                                                        value_name="unit_solds")
                     .assign(simple_trend=lambda df: df["date"].str.extract(r"(\d+)").astype(int))
                    )

In [30]:
df_all_features.tail(5)[features+["d"]]

,simple_trend,wday,month,snap_CA,snap_TX,snap_WI,sell_price,item_id_enc,dept_id_enc,store_id_enc,state_id_enc,cat_id_enc,event_name_1_enc,event_name_2_enc,event_type_1_enc,event_type_2_enc,d
58327365,1913,2,4,0,0,0,2.980469,1432,2,9,2,0,18,3,1,1,d_1913
58327366,1913,2,4,0,0,0,2.480469,1433,2,9,2,0,18,3,1,1,d_1913
58327367,1913,2,4,0,0,0,3.980469,1434,2,9,2,0,18,3,1,1,d_1913
58327368,1913,2,4,0,0,0,1.280273,1435,2,9,2,0,18,3,1,1,d_1913
58327369,1913,2,4,0,0,0,1.000000,1436,2,9,2,0,18,3,1,1,d_1913


In [32]:
df_calendar.tail()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0
1968,2016-06-19,11621,Sunday,2,6,2016,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0


In [35]:
df_all_features["simple_trend"].max()

1913

In [39]:
df_sample_melted["simple_trend"] += df_sales_melted["simple_trend"].max()
df_sample_melted["d"] = ("d_" + df_sample_melted["simple_trend"].astype(str))

In [40]:
df_sample_melted.head(5)

,id,item_id,dept_id,store_id,state_id,cat_id,date,unit_solds,simple_trend,d
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,CA_1,CA,HOBBIES,F1,0,1914,d_1914
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,CA_1,CA,HOBBIES,F1,0,1914,d_1914
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,CA_1,CA,HOBBIES,F1,0,1914,d_1914
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,CA_1,CA,HOBBIES,F1,0,1914,d_1914
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,CA_1,CA,HOBBIES,F1,0,1914,d_1914


In [41]:
df_sample_cal = (df_sample_melted.merge(df_calendar,
                                        on="d",
                                        how="left"))

In [66]:
df_sample_cal[["item_id", "store_id", "wm_yr_wk"]].head(5)

,item_id,store_id,wm_yr_wk
0,HOBBIES_1_001,CA_1,11613
1,HOBBIES_1_002,CA_1,11613
2,HOBBIES_1_003,CA_1,11613
3,HOBBIES_1_004,CA_1,11613
4,HOBBIES_1_005,CA_1,11613


In [69]:
df_sample_all_features[df_sample_all_features["wm_yr_wk"]==11613][features].head(5)

,simple_trend,wday,month,snap_CA,snap_TX,snap_WI,sell_price,item_id_enc,dept_id_enc,store_id_enc,state_id_enc,cat_id_enc,event_name_1_enc,event_name_2_enc,event_type_1_enc,event_type_2_enc
0,1914,3,4,0,0,0,8.382812,1437,3,0,0,1,2,0,1,0
1,1914,3,4,0,0,0,3.970703,1438,3,0,0,1,2,0,1,0
2,1914,3,4,0,0,0,2.970703,1439,3,0,0,1,2,0,1,0
3,1914,3,4,0,0,0,4.640625,1440,3,0,0,1,2,0,1,0
4,1914,3,4,0,0,0,2.880859,1441,3,0,0,1,2,0,1,0


In [43]:
df_sample_all_features = (df_sample_cal
                           .merge(df_prices, 
                                  on=["item_id", "store_id", "wm_yr_wk"], 
                                  how="left")
                          .pipe(reduce_mem_usage))
df_sample_all_features.shape

Mem. usage decreased to 218.20 Mb (33.0% reduction)


(1707440, 24)

In [44]:
df_sample_all_features.head(5)

,id,item_id,dept_id,store_id,state_id,cat_id,date_x,unit_solds,simple_trend,d,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,CA_1,CA,HOBBIES,F1,0,1914,d_1914,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,8.382812
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,CA_1,CA,HOBBIES,F1,0,1914,d_1914,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,3.970703
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,CA_1,CA,HOBBIES,F1,0,1914,d_1914,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,2.970703
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,CA_1,CA,HOBBIES,F1,0,1914,d_1914,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,4.640625
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,CA_1,CA,HOBBIES,F1,0,1914,d_1914,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,2.880859


In [70]:
for categorical_col in categorical_cols:
    if "event" in categorical_col:
        df_sample_all_features[categorical_col] = df_sample_all_features[categorical_col].fillna("NULL")
        
    lb = LabelEncoder()
    lb.fit(df_all_features[categorical_col])
    fname = f"{categorical_col}_enc"
    df_sample_all_features[fname] = lb.transform(df_sample_all_features[categorical_col])

In [71]:
sample_forecast = lgmb_clf.predict(df_sample_all_features[features])

In [72]:
df_sample_all_features['forecast'] = sample_forecast

In [73]:
df_submission = df_sample_all_features.pivot(index="id", columns="date_x", values="forecast")

In [74]:
df_sample_all_features.head(4)

,id,item_id,dept_id,store_id,state_id,cat_id,date_x,unit_solds,simple_trend,d,...,item_id_enc,dept_id_enc,store_id_enc,state_id_enc,cat_id_enc,event_name_1_enc,event_name_2_enc,event_type_1_enc,event_type_2_enc,forecast
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,CA_1,CA,HOBBIES,F1,0,1914,d_1914,...,1437,3,0,0,1,18,3,1,1,0.713825
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,CA_1,CA,HOBBIES,F1,0,1914,d_1914,...,1438,3,0,0,1,18,3,1,1,0.713297
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,CA_1,CA,HOBBIES,F1,0,1914,d_1914,...,1439,3,0,0,1,18,3,1,1,0.874190
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,CA_1,CA,HOBBIES,F1,0,1914,d_1914,...,1440,3,0,0,1,18,3,1,1,0.716601


In [75]:
df_submission.to_csv("../data/submission/raw_submission_all_features.csv")